In [1]:
!pip install tiktoken datasets tqdm

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken
import numpy as np
import os
import requests
import math
print(torch.cuda.device_count())

# Hyperparameters
batch_size = 32  # How many batches per training step
context_length = 256  # Length of the token chunk each batch
d_model = 512  # The size of our model token embeddings
num_blocks = 16  # Number of transformer blocks
num_heads = 8  # Number of heads in Multi-head attention
learning_rate = 3.142e-4  # 0.001
dropout = 0.1  # Dropout rate
max_iters = 50000  # Total of training iterations <- Change this to smaller number for testing
eval_interval = 30  # How often to evaluate
eval_iters = 20  # Number of iterations to average for evaluation
grad_clip = 1.0
warmup_steps = 2000
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if it's available.
TORCH_SEED = 1337
torch.manual_seed(TORCH_SEED)

2


In [12]:
from datasets import load_dataset
import torch
import tiktoken

print("Loading dataset (streaming)...")

dataset = load_dataset(
    "HuggingFaceFW/fineweb-edu",
    split="train",
    streaming=True
)

enc = tiktoken.get_encoding("cl100k_base")
max_token_val = enc.n_vocab
tokens = []

LIMIT = 15_000_000   # number of tokens you want

print("Tokenizing...")

total = 0

for example in dataset:
    text = example["text"]

    if text.strip():
        ids = enc.encode(text)
        tokens.extend(ids + [enc.eot_token])
        total += len(ids)

    if total >= LIMIT:
        break

data = torch.tensor(tokens, dtype=torch.long)

# split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print("Train tokens:", len(train_data))
print("Val tokens:", len(val_data))

Loading dataset (streaming)...


Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

Tokenizing...
Train tokens: 13525801
Val tokens: 1502867


In [ ]:
from datasets import load_dataset
import torch
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")
max_token_val = enc.n_vocab
tokens = []

# Streaming OpenAssistant dataset
dataset = load_dataset("OpenAssistant/oasst1", "default", split="train", streaming=True)

LIMIT = 15_000_000
total = 0

for ex in dataset:
    # Use the 'text' field directly
    text = ex.get("text", "")
    if text.strip():
        ids = enc.encode(text)
        tokens.extend(ids + [enc.eot_token])
        total += len(ids)

    if total >= LIMIT:
        break

data = torch.tensor(tokens, dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print("Train tokens:", len(train_data))
print("Val tokens:", len(val_data))

In [3]:
class FFN(nn.Module):
    def __init__(self):
        super().__init__()
        self.d_model = d_model
        self.dropout = dropout
        self.ffn = nn.Sequential(
            nn.Linear(in_features=self.d_model, out_features=self.d_model * 4),
            nn.ReLU(),
            nn.Linear(in_features=self.d_model * 4, out_features=self.d_model * 4),
            nn.Linear(in_features=self.d_model * 4, out_features=self.d_model),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.ffn(x)
    
class Attention(nn.Module):
    def __init__(self, head_size: int):
        super().__init__()
        self.d_model = d_model
        self.head_size = head_size
        self.context_length = context_length
        self.dropout = dropout

        self.k = nn.Linear(in_features=self.d_model, out_features=self.head_size, bias=False)
        self.v = nn.Linear(in_features=self.d_model, out_features=self.head_size, bias=False)
        self.q = nn.Linear(in_features=self.d_model, out_features=self.head_size, bias=False)
        self.register_buffer('tril', torch.tril(
            torch.ones((self.context_length, self.context_length))))
        self.dropout_layer = nn.Dropout(self.dropout)

    def forward(self, x):
        B,T,C = x.shape
        assert T <= self.context_length
        assert C == self.d_model
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        # scaled dot prod
        weights = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        # apply mask
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        weights = F.softmax(input=weights, dim=-1)
        weights = self.dropout_layer(weights)

        out = weights @ v
        return out
    
class MHA(nn.Module):
    def __init__(self, head_size: int):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.d_model = d_model
        self.context_length = context_length
        self.dropout = dropout

        self.heads = nn.ModuleList([Attention(head_size=self.head_size) for _ in range(self.num_heads)])
        self.projection_layer = nn.Linear(in_features=self.d_model, out_features=self.d_model)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.projection_layer(out)
        out = self.dropout_layer(out)
        return out
def precompute_rope_freqs(head_dim, max_seq_len):
    assert head_dim % 2 == 0, "head_dim must be even"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    theta = 1.0 / (10000 ** (
        torch.arange(0, head_dim, 2, device=device).float() / head_dim
    ))

    positions = torch.arange(max_seq_len, device=device).float()

    freqs = torch.outer(positions, theta)  # (T, head_dim/2)

    return freqs
def apply_rope(x, freqs):
    # x shape: (B, T, H, D)

    B, T, H, D = x.shape

    # Move to (B, H, T, D)
    x = x.permute(0, 2, 1, 3)

    freqs = freqs[:T]  # (T, D/2)

    # reshape for broadcasting
    freqs = freqs.unsqueeze(0).unsqueeze(0)  # (1,1,T,D/2)

    x1, x2 = x[..., :D//2], x[..., D//2:]

    x = torch.cat([
        x1 * freqs.cos() - x2 * freqs.sin(),
        x1 * freqs.sin() + x2 * freqs.cos()
    ], dim=-1)

    # return to original shape
    return x.permute(0, 2, 1, 3)

class GQAAttention(nn.Module):
    def __init__(self, head_size: int):
        super().__init__()

        self.n_q = num_heads           # query heads
        self.n_kv = num_heads // 4       # KV heads (smaller)
        self.head_dim = d_model // self.n_q

        self.q_proj = nn.Linear(d_model, self.n_q * self.head_dim)
        self.k_proj = nn.Linear(d_model, self.n_kv * self.head_dim)
        self.v_proj = nn.Linear(d_model, self.n_kv * self.head_dim)

        self.out_proj = nn.Linear(d_model, d_model)

        self.register_buffer(
            "rope_freqs",
            precompute_rope_freqs(self.head_dim, context_length),
            persistent=False,
        )

    def forward(self, x):
        B, T, C = x.shape

        q = self.q_proj(x).view(B, T, self.n_q, self.head_dim)
        k = self.k_proj(x).view(B, T, self.n_kv, self.head_dim)
        v = self.v_proj(x).view(B, T, self.n_kv, self.head_dim)

        # Apply RoPE
        q = apply_rope(q, self.rope_freqs)
        k = apply_rope(k, self.rope_freqs)

        # Expand KV to match Q groups
        repeat = self.n_q // self.n_kv
        k = k.repeat_interleave(repeat, dim=2)
        v = v.repeat_interleave(repeat, dim=2)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        out = F.scaled_dot_product_attention(
            q, k, v, is_causal=True
        )

        out = out.transpose(1, 2).contiguous().view(B, T, C)
        return self.out_proj(out)

class RMSNorm(nn.Module):
    """Root Mean Square Layer Normalization - more stable than LayerNorm"""
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        # x: (B, T, C)
        norm = x.norm(2, dim=-1, keepdim=True) * (x.size(-1) ** -0.5)
        return self.weight * x / (norm + self.eps)

class TB(nn.Module):
    def __init__(self, num_heads: int):
        super().__init__()
        self.d_model = d_model
        self.context_length = context_length
        self.head_size = d_model // num_heads
        self.nums_heads = num_heads
        self.dropout = dropout

        # self.mha = MHA(head_size=self.head_size)
        self.mha = GQAAttention(head_size=self.head_size)
        self.ffn = FFN()
        self.norm1 = RMSNorm(self.d_model)
        self.norm2 = RMSNorm(self.d_model)
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

    def forward(self, x):
        x = x + self.mha(self.norm1(x))
        x = x + self.ffn(self.norm2(x))
        return x
    
class LModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.d_model = d_model
        self.context_length = context_length
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.dropout = dropout
        self.max_token_value = max_token_val

        self.token_em = nn.Embedding(num_embeddings=self.max_token_value, embedding_dim=self.d_model)
        self.transformer_blocks = nn.Sequential(*(
            [TB(num_heads=self.num_heads) for _ in range(self.num_blocks)] + [nn.LayerNorm(self.d_model)]
        ))
        self.lmoll = nn.Linear(in_features=self.d_model, out_features=self.max_token_value + 1)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        pelt = torch.zeros(self.context_length, self.d_model)
        pos = torch.arange(0, self.context_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.d_model, 2).float() * (-math.log(10000.0)/self.d_model))
        pelt[:, 0::2] = torch.sin(pos * div_term)
        pelt[:, 1::2] = torch.cos(pos * div_term)

        pos_em = pelt[:T, :].to(device)
        x = self.token_em(idx) + pos_em
        x = self.transformer_blocks(x)

        logits = self.lmoll(x)
        if targets is not None:
            B, T, C = logits.shape
            logits_reshaped = logits.view(B*T, C)
            targets_reshaped = targets.view(B*T)
            loss = F.cross_entropy(input=logits_reshaped, target=targets_reshaped)
        else:
            loss = None
        return logits, loss
    def generate(self, idx, max_new_token, temperature=1.0, top_k=None):
        for _ in range(max_new_token):
            idx_corp = idx[:, -self.context_length:]
            logits, _ = self(idx_corp)

            logits = logits[:, -1, :] / temperature

            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float("inf")

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, 1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx
    
model = LModel()
model = model.to(device)

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = torch.nn.DataParallel(model)

total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total params: {total:,}")
print(f"Trainable params: {trainable:,}")

# Get input embedding batch
def get_batch(split: str):
    data = train_data if split == 'train' else val_data
    idxs = torch.randint(low=0, high=len(data) - context_length, size=(batch_size,))
    x = torch.stack([data[idx:idx + context_length] for idx in idxs])
    y = torch.stack([data[idx + 1:idx + context_length + 1] for idx in idxs])
    return x.to(device), y.to(device)
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'valid']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x_batch, y_batch = get_batch(split)
            with torch.amp.autocast('cuda'):
                logits, loss = model(x_batch, y_batch)
                # Handle DataParallel returning tensor with multiple elements
                if isinstance(loss, torch.Tensor) and loss.numel() > 1:
                    loss = loss.mean()
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Using 2 GPUs
Total params: 214,045,110
Trainable params: 214,045,110


In [7]:
import os

# ckpt_path = "/kaggle/input/datasets/yashrajrao/weights/200M.pt"
# ckpt_path = "/kaggle/working/model-ckpt-best.pt"
ckpt_path = "/kaggle/input/datasets/yashrajrao/w2-update/214M-4.6.pt"

start_step = 0

if os.path.exists(ckpt_path):
    print("Loading pretrained checkpoint...")

    state_dict = torch.load(ckpt_path, map_location=device)

    # Handle DataParallel case
    if hasattr(model, "module"):
        model.module.load_state_dict(state_dict)
    else:
        model.load_state_dict(state_dict)

    print("Checkpoint loaded successfully!")
else:
    print("No checkpoint found. Training from scratch.")

Loading pretrained checkpoint...
Checkpoint loaded successfully!


In [9]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [6]:
if __name__ == "__main__":
    # Optimizer with weight decay
    optimizer = torch.optim.AdamW(
        params=model.parameters(), 
        lr=learning_rate, 
        betas=(0.9, 0.95),
        weight_decay=0.02
    )
    
    # Learning rate scheduler with warmup
    def get_lr(step):
        if step < warmup_steps:
            return learning_rate * step / warmup_steps
        return learning_rate * 0.5 * (1 + math.cos(math.pi * (step - warmup_steps) / (max_iters - warmup_steps)))
    
    # Mixed precision scaler (updated syntax)
    scaler = torch.amp.GradScaler('cuda')
    
    tracked_losses = list()
    best_val_loss = float('inf')
    
    for step in range(max_iters):
        # LR scheduling
        lr = get_lr(step)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        
        # Evaluation
        if step % eval_interval == 0 or step == max_iters - 1:
            losses = estimate_loss()
            tracked_losses.append(losses)
            print(f"Step: {step} | LR: {lr:.2e} | Train Loss: {losses['train'].item():.4f} | Val Loss: {losses['valid'].item():.4f}")
            
            # Save best model (handle DataParallel)
            if losses['valid'].item() < best_val_loss:
                best_val_loss = losses['valid'].item()
                # Save module state_dict to avoid DataParallel wrapper issues
                state_dict = model.module.state_dict() if hasattr(model, 'module') else model.state_dict()
                torch.save(state_dict, 'model-ckpt-best.pt')
                print(f"Saved best model with val loss: {best_val_loss:.4f}")
        
        # Training step with mixed precision
        xb, yb = get_batch('train')
        xb, yb = xb.to(device), yb.to(device)
        
        optimizer.zero_grad(set_to_none=True)
        
        with torch.amp.autocast('cuda'):
            logits, loss = model(xb, yb)
            # Average loss across GPUs if using DataParallel
            if isinstance(loss, torch.Tensor) and loss.numel() > 1:
                loss = loss.mean()
        
        scaler.scale(loss).backward()
        
        # Gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        
        scaler.step(optimizer)
        scaler.update()
        
        # Memory cleanup
        if step % 100 == 0:
            torch.cuda.empty_cache()
    
    # Save final (handle DataParallel)
    state_dict = model.module.state_dict() if hasattr(model, 'module') else model.state_dict()
    torch.save(state_dict, 'model-ckpt-final.pt')
    print(f"Training complete. Best val loss: {best_val_loss:.4f}")

/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step: 0 | LR: 0.00e+00 | Train Loss: 8.7864 | Val Loss: 7.6828
Saved best model with val loss: 7.6828
Step: 30 | LR: 4.71e-06 | Train Loss: 8.3293 | Val Loss: 7.6765
Saved best model with val loss: 7.6765
Step: 60 | LR: 9.43e-06 | Train Loss: 7.9659 | Val Loss: 7.1912
Saved best model with val loss: 7.1912
Step: 90 | LR: 1.41e-05 | Train Loss: 7.8366 | Val Loss: 7.1345
Saved best model with val loss: 7.1345
Step: 120 | LR: 1.89e-05 | Train Loss: 7.7805 | Val Loss: 6.9997
Saved best model with val loss: 6.9997
Step: 150 | LR: 2.36e-05 | Train Loss: 7.3519 | Val Loss: 6.9202
Saved best model with val loss: 6.9202
Step: 180 | LR: 2.83e-05 | Train Loss: 7.1841 | Val Loss: 6.5594
Saved best model with val loss: 6.5594
Step: 210 | LR: 3.30e-05 | Train Loss: 6.9305 | Val Loss: 6.4334
Saved best model with val loss: 6.4334
Step: 240 | LR: 3.77e-05 | Train Loss: 6.6750 | Val Loss: 6.3424
Saved best model with val loss: 6.3424
Step: 270 | LR: 4.24e-05 | Train Loss: 6.5533 | Val Loss: 6.2369
Save

KeyboardInterrupt: 

In [9]:
from IPython.display import HTML
import shutil

download_link = f'<a href="file:///kaggle/working/model-ckpt-best.pt" download="model-ckpt-best.pt">Download Model</a>'
HTML(download_link)

In [8]:
model.eval()
start = 'The world'
start_ids = enc.encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
y = model.module.generate(x, max_new_token=100)
print('---------------')
print(enc.decode(y[0].tolist()))
print('---------------')

---------------
The world of Arist sine and heroes is responsible foroard in their work:

1. Find a manteca and explain why they are still using such software.

2. In Once plants and animals are through their ability to automate and reasonable control, cut them some of them.

3. The night of their operating system and image is moving back towards their ownastr and att Design.

4. They miscrc, affecting the complex project, spending risks and representation from all animals in the world.

5. In addition,
---------------
